In [1]:
import json
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import random
import torch
from torch.optim import Adam
import dgl
from datetime import datetime

from src import *
from pprint import pprint
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

nclient = 2

data_dir = './data_passrefinder-p'
output_dir = f"{data_dir}/output/fl_passrefinder/client_{nclient}/{current_time}"

In [2]:
from argparse import ArgumentParser
parser = ArgumentParser()

import argparse
from pprint import pprint

parser = argparse.ArgumentParser(description="Parser for model configuration")

parser.add_argument('--feature_file', type=str, default=f"{data_dir}/data/feature_dict.json", help='feature json file')
parser.add_argument('--reuse_rate_file', type=str, default=f"{data_dir}/data/reuse_rate_dict.json", help='reuse rate json file')
# parser.add_argument('--setting', type=str, required=True, help='graph learning setting: inductive/transductive')
# parser.add_argument('--model_path', type=str, required=True, default="./model/", help='model file path')
parser.add_argument('--output_dir', type=str, default=output_dir, help='output path')
parser.add_argument('--agg_type', type=str, default="no_hidden", help='aggregation type')
parser.add_argument('--nclient', type=int, default=nclient, help='number of clients')
# parser.add_argument('--valid', type=float, default=0.2, help='split ratio of validation set')
# parser.add_argument('--test', type=float, default=0.2, help='split ratio of test set')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout ratio')
parser.add_argument('--random_seed', type=int, default=42, help='random seed for initialization')
parser.add_argument('--relu', type=float, default=0.2, help='ReLU threshold')
parser.add_argument('--reuse_th', type=float, default=0.5, help='threshold for reuse')
parser.add_argument('--train_batch_size', type=int, default=2048, help='batch size for training')
parser.add_argument('--eval_batch_size', type=int, default=2048, help='batch size for evaluation')
parser.add_argument('--embed_size', type=int, default=1024, help='size of the embedding layer')
parser.add_argument('--hidden_size', type=int, default=1024, help='size of the hidden layer')
parser.add_argument('--gnn_depth', type=int, default=2, help='depth of the GNN')
parser.add_argument('--max_lr', type=float, default=0.001, help='maximum learning rate')
parser.add_argument('--warmup', type=float, default=0.02, help='warmup ratio for learning rate')
parser.add_argument('--max_epoch', type=int, default=100, help='maximum number of epochs')
# parser.add_argument('--early_stop', type=int, default=40, help='early stopping epochs')
parser.add_argument('--device', type=int, default=0, help='GPU ID')

args = parser.parse_args(args=[])
pprint(vars(args))

torch.manual_seed(args.random_seed)
torch.cuda.manual_seed(args.random_seed)
dgl.seed(args.random_seed)
np.random.seed(args.random_seed)
random.seed(args.random_seed)

{'agg_type': 'no_hidden',
 'device': 0,
 'dropout': 0.5,
 'embed_size': 1024,
 'eval_batch_size': 2048,
 'feature_file': './data_passrefinder-p/data/feature_dict.json',
 'gnn_depth': 2,
 'hidden_size': 1024,
 'max_epoch': 100,
 'max_lr': 0.001,
 'nclient': 2,
 'output_dir': './data_passrefinder-p/output/fl_passrefinder/client_2/2025-09-01 '
               '20:45:15',
 'random_seed': 42,
 'relu': 0.2,
 'reuse_rate_file': './data_passrefinder-p/data/reuse_rate_dict.json',
 'reuse_th': 0.5,
 'train_batch_size': 2048,
 'warmup': 0.02}


In [3]:
with open(args.feature_file, 'r') as f:
    feature_dict = json.load(f)

with open(args.reuse_rate_file, 'r') as f:
    reuse_rate_dict = json.load(f)
    


g = construct_graph(feature_dict, reuse_rate_dict, args.reuse_th)
g_dict, node_set_list, target_eids_dict = graph_split_FL(g, args.nclient, args.random_seed)



# train_loader_list = []
# for g, nodes in zip(g_split['train'], node_split['train']):
#     train_loader_list.append(get_data_loader(g, nodes, args.train_batch_size, args.gnn_depth, suffle=True))
# train_nfeat_list = P.pop_train_node_feature()

# valid_loader = get_data_loader(g_split['valid'], node_split['valid'])
# valid_nfeat = P.pop_node_feature('valid')
# test_loader = get_data_loader(g_split['test'], node_split['test'])
# test_nfeat = P.pop_node_feature('test')


P = PassREfinder_FL(g_dict, node_set_list, target_eids_dict, args)
# P.print_graph()

Constructing graph...
Constructing graph... Done
Splitting graph...
Splitting graph... Done


In [4]:
pprint(g_dict)

{'test': Graph(num_nodes={'website': 9927},
      num_edges={('website', 'reuse', 'website'): 1211942, ('website', 'user', 'website'): 0},
      metagraph=[('website', 'website', 'reuse'), ('website', 'website', 'user')]),
 'train': [Graph(num_nodes={'website': 1000},
      num_edges={('website', 'reuse', 'website'): 48990, ('website', 'user', 'website'): 0},
      metagraph=[('website', 'website', 'reuse'), ('website', 'website', 'user')]),
           Graph(num_nodes={'website': 1000},
      num_edges={('website', 'reuse', 'website'): 14484, ('website', 'user', 'website'): 0},
      metagraph=[('website', 'website', 'reuse'), ('website', 'website', 'user')])],
 'valid': Graph(num_nodes={'website': 9927},
      num_edges={('website', 'reuse', 'website'): 112316, ('website', 'user', 'website'): 0},
      metagraph=[('website', 'website', 'reuse'), ('website', 'website', 'user')])}


In [5]:
os.makedirs(args.output_dir, exist_ok=True)

with open(os.path.join(args.output_dir, 'arguments.json'), 'w') as f:
    json.dump(vars(args), f)

# args.device = torch.device(f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu')

P.train()

2025-09-01 20:45:25 start


100%|██████████| 24/24 [00:05<00:00,  4.49it/s]

2025-09-01 20:45:37 Epoch [1/100], Train Loss: 0.7739, Valid Loss: 0.6946, Precision: 0.5316, Recall: 0.8632, F1: 0.6580


Save model!


100%|██████████| 24/24 [00:04<00:00,  5.44it/s]


...

2025-09-01 21:01:23 Epoch [98/100], Train Loss: 0.5305, Valid Loss: 0.4974, Precision: 0.8146, Recall: 0.6895, F1: 0.7468



100%|██████████| 24/24 [00:04<00:00,  5.30it/s]


2025-09-01 21:01:32 Epoch [99/100], Train Loss: 0.5384, Valid Loss: 0.4974, Precision: 0.8110, Recall: 0.6917, F1: 0.7466


100%|██████████| 24/24 [00:04<00:00,  5.61it/s]


2025-09-01 21:01:42 Epoch [100/100], Train Loss: 0.5275, Valid Loss: 0.4964, Precision: 0.8141, Recall: 0.6913, F1: 0.7477


In [6]:
%%time
P.model = load_model(os.path.join(P.args.output_dir, 'model.pt'), P.model)
result = P.evaluate()
f1 = result['eval_f1']
precision = result['eval_precision']
recall = result['eval_recall']
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

with open(os.path.join(args.output_dir, 'eval_results.json'), 'w') as f:
    json.dump(result, f)

Evaluating...


  0%|          | 0/511 [00:00<?, ?it/s]/home/jaehan/research/passrefinder-p/src/trainer.py:268: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score_values = F.softmax(batch_pred.detach().cpu())[:,1].numpy().tolist()
100%|██████████| 511/511 [07:21<00:00,  1.16it/s]


Precision: 0.7877, Recall: 0.4545, F1: 0.5764
CPU times: user 24min 53s, sys: 5.38 s, total: 24min 58s
Wall time: 7min 23s
